# Deep learning
This notebook is responsible for implementing a recurrent neural network using TensorFlow.

## Database credentials

In [2]:
db_user = ""
db_pass = ""
db_name = ""
db_host = "localhost"
with open("database_credentials.txt") as f:
    db_user = f.readline().strip()
    db_pass = f.readline().strip()
    db_name = f.readline().strip()

## Dataframe-ize tweets

In [3]:
import MySQLdb as mdb
import numpy as np
import pandas as pd

In [4]:
try:
    con = mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
    df = pd.read_sql("""SELECT * FROM search_tweets""", con)
finally:
    if con:
        con.close()
df.head()

,id,message
0,1,Great meeting with @Cabinet at the @WhiteHouse...
1,2,Looking forward to 3:30 P.M. meeting today at ...
2,3,"Lowest rated Oscars in HISTORY. Problem is, we..."
3,4,"JOBS, JOBS, JOBS! #MAGA"
4,5,The U.S. is acting swiftly on Intellectual Pro...


## Data exploration

In [24]:
#All tweets linearly joined together in a list
char_list = " ".join(list(df["message"]))

print("Unique space-separated character orderings: {}".format(len({
    word: None for word in char_list.split(" ")})))
print("Tweets: {}".format(df.shape[0]))

print("Average sentences per tweet: {}".format(
    (char_list.count(".") + char_list.count("?") + char_list.count("!")) / float(df.shape[0])))

Unique space-separated character orderings: 12837
Tweets: 2889
Average sentences per tweet: 2.76081689166


## Data preprocessing
### Lookup table
In order to create a word embedding, the words used in the tweets need to be transformed to IDs. The 2 way mapping from words->IDs and IDs->words is generated below.

In [26]:
from string import punctuation
from collections import Counter

In [46]:
sample_text = "here's some sample text. hopefully this\nworks? ok - time to give it a shot!!"
def get_lookup_tables(text):
    """
    Gets the lookup tables mapping character orderings to their IDs and vice-versa.
    :param text: Text to create lookup tables from
    :return: A tuple of mapes (vocab_to_int, int_to_vocab)
    """
    #If passed in text as big string, words separated by spaces
    if type(text) == str:
        #text = text.translate(None, punctuation).split()
        text = text.split()
    #If passed in text as list (same as string representation but separated by indices)
    elif type(text) == list:
        #Handle later
        None
        
    #Create mappings
    words = [k for (k,v) in Counter(text).items()]
    vocab_to_int = {}
    int_to_vocab = {}
    for i in range(len(words)):
        vocab_to_int[words[i]] = i
        int_to_vocab[i] = words[i]
    return (vocab_to_int, int_to_vocab)
#get_lookup_tables(sample_text)

## Punctuation tokenizing
Spaces split the tweets up word by words. However, punctuations make it difficult for neural networks to distinguish between "dream" and "dream!". The requring tokenization mechanism to map characters to their IDs is performed below.

With this mapping mechanism, the dictionary will be used to toeknize the symbols and add a space around the character, making the character it's own word. When punctuations act as their own word, the neural network can more easily incorporate them into it's produced language.

In [50]:
#Consider adding possessive/abbreviation for punctuation ... "'"
rnn_punctuation = [".", ",", "\"", ";", "!", "?", "(", ")", "-", "\n", "|"]
rnn_punctuation_words = map(lambda s : "~" + s.upper() + "~", ["period", "comma", "quotation", "semicolon", "exclamation",
                         "question", "leftparen", "rightparen", "hyphen", "newline", "pipe"])

rnn_punctuation_map = {rnn_punctuation[i]: rnn_punctuation_words[i] for i in range(len(rnn_punctuation))}
rnn_punctuation_map

{'\n': '~NEWLINE~',
 '!': '~EXCLAMATION~',
 '"': '~QUOTATION~',
 '(': '~LEFTPAREN~',
 ')': '~RIGHTPAREN~',
 ',': '~COMMA~',
 '-': '~HYPHEN~',
 '.': '~PERIOD~',
 ';': '~SEMICOLON~',
 '?': '~QUESTION~',
 '|': '~PIPE~'}